<h1 style="
  background: linear-gradient(to right,rgb(0, 0, 108),rgb(159, 207, 255));
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  font-size: 38px;
  font-weight: bold;
">
  7. Máquina de Soporte Vectorial
</h1>